In [1]:
from utz import *

In [2]:
from nj_crashes.sri.sri import get_sris
sris = get_sris()
len(sris)

34479

In [24]:
%%time
from njdot.data import START_YEAR, END_YEAR
c = concat([
    read_parquet(f'data/{year}/NewJersey{year}Accidents.pqt')
    for year in range(START_YEAR, END_YEAR)
]).reset_index(drop=True)
c

CPU times: user 9.98 s, sys: 5.01 s, total: 15 s
Wall time: 14 s


,County Code,Municipality Code,Department Case Number,County Name,Municipality Name,Police Department Code,Police Department,Police Station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,Severity,Intersection,Alcohol Involved,HazMat Involved,Crash Type Code,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,SRI (Standard Route Identifier),Mile Post,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,Latitude,Longitude,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,Date,Road Horizontal Alignment,Road Grade,First Harmful Event
0,01,01,#2001-17846,ATLANTIC,ABSECON CITY,01,ABSECON CITY,MUNICIPAL COMP?,0,0,0,0,P,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN
1,01,01,01-00029,ATLANTIC,ABSECON CITY,1,ABSECON,,0,0,0,0,P,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN
2,01,01,01-004615,ATLANTIC,ABSECON CITY,1,ABSECON,,0,4,0,0,I,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN
3,01,01,01-004880,ATLANTIC,ABSECON CITY,1,ABSECON,,0,1,0,0,I,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN
4,01,01,01-004912,ATLANTIC,ABSECON CITY,1,ABSECON,,0,0,0,0,P,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,21,23,B150-2021-00117A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46
6077299,21,23,B150-2021-00118A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,75.05549,N,,8610,2021-05-25 07:26:00,01,04,26
6077300,21,23,B150-2021-00128A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,75.07627,N,,8514,2021-06-03 20:17:00,03,04,24
6077301,21,23,B150-2021-00130A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,US 46,E,46,,00000046__,7.50,02,,02,01,07,01,05,01,,,,,,,,50,,40.84149,75.04968,N,,8365,2021-06-06 00:00:00,01,04,24


In [4]:
sri = c['SRI (Standard Route Identifier)'].rename('sri')
mp = c['Mile Post'].rename('mp')

In [5]:
from nj_crashes.sri.sri import LL

In [6]:
from dataclasses import asdict

In [7]:
def geocode_mp(r):
    sri = r.sri
    mp = r.mp
    if isna(sri):
        return dict(reason="No SRI")
    if isna(mp):
        return dict(reason="No MP")
    if sri not in sris:
        return dict(reason='SRI not found')
    mp_lls = sris[sri]
    ll = mp_lls.get(mp)
    if ll:
        return asdict(ll)
    else:
        return dict(reason="MP didn't geocode")

In [17]:
%%time
geocoded_mps = DF(sxs(sri, mp).apply(geocode_mp, axis=1).tolist())
geocoded_mps

CPU times: user 59.6 s, sys: 1.19 s, total: 1min
Wall time: 1min 1s


,reason,lon,lat
0,No MP,NaN,NaN
1,No MP,NaN,NaN
2,No MP,NaN,NaN
3,NaN,-74.512308,39.432705
4,No MP,NaN,NaN
...,...,...,...
6077298,NaN,-75.075607,40.807156
6077299,NaN,-75.056122,40.844345
6077300,NaN,-75.015149,40.863108
6077301,NaN,-75.046404,40.840421


In [19]:
geocoded_mps.reason.value_counts(dropna=False)

NaN                  3479827
No MP                2367547
MP didn't geocode     141019
SRI not found          88910
Name: reason, dtype: int64

In [21]:
geocoded_mps.isna().sum()

reason    3479827
lon       2597476
lat       2597476
dtype: int64

In [25]:
lat = c.Latitude
lon = c.Longitude
#ll = sxs(lon, lat)
ll = sxs(geocoded_mps, lon, lat)
ll

,reason,lon,lat,Longitude,Latitude
0,No MP,NaN,NaN,NaN,NaN
1,No MP,NaN,NaN,NaN,NaN
2,No MP,NaN,NaN,NaN,NaN
3,NaN,-74.512308,39.432705,NaN,NaN
4,No MP,NaN,NaN,NaN,NaN
...,...,...,...,...,...
6077298,NaN,-75.075607,40.807156,75.07559,40.80707
6077299,NaN,-75.056122,40.844345,75.05549,40.84466
6077300,NaN,-75.015149,40.863108,75.07627,40.80564
6077301,NaN,-75.046404,40.840421,75.04968,40.84149


In [28]:
pd.crosstab(
    (~ll.lon.isna()).rename('has lon'),
    (~ll.Longitude.isna()).rename('has Longitude'),
)

has Longitude,False,True
has lon,,
False,2504641,92835
True,2317307,1162520


In [27]:
has_rsn = ~ll.reason.isna()
has_ll = ~ll.Latitude.isna()
print(f'{has_rsn.sum()} crashes failed to geocode based on SRI / Mile Post')
print(f'{has_ll.sum()} crashes have Latitude/Longitude')
print(f'{(has_rsn & has_ll).sum()} crashes have only have Latitude/Longitude')
print
(~(ll.reason.isna()) | ll.Latitude.isna()).sum()

4914783

In [29]:
cg = sxs(c, geocoded_mps)
cg

,County Code,Municipality Code,Department Case Number,County Name,Municipality Name,Police Department Code,Police Department,Police Station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,Severity,Intersection,Alcohol Involved,HazMat Involved,Crash Type Code,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,SRI (Standard Route Identifier),Mile Post,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,Latitude,Longitude,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,Date,Road Horizontal Alignment,Road Grade,First Harmful Event,reason,lon,lat
0,01,01,#2001-17846,ATLANTIC,ABSECON CITY,01,ABSECON CITY,MUNICIPAL COMP?,0,0,0,0,P,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,No MP,NaN,NaN
1,01,01,01-00029,ATLANTIC,ABSECON CITY,1,ABSECON,,0,0,0,0,P,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,No MP,NaN,NaN
2,01,01,01-004615,ATLANTIC,ABSECON CITY,1,ABSECON,,0,4,0,0,I,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,No MP,NaN,NaN
3,01,01,01-004880,ATLANTIC,ABSECON CITY,1,ABSECON,,0,1,0,0,I,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,NaN,-74.512308,39.432705
4,01,01,01-004912,ATLANTIC,ABSECON CITY,1,ABSECON,,0,0,0,0,P,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,No MP,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,21,23,B150-2021-00117A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,NaN,-75.075607,40.807156
6077299,21,23,B150-2021-00118A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,NaN,-75.056122,40.844345
6077300,21,23,B150-2021-00128A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,NaN,-75.015149,40.863108
6077301,21,23,B150-2021-00130A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,US 46,E,46,,00000046__,7.50,02,,02,01,07,01,05,01,,,,,,,,50,,40.84149,75.04968,N,,8365,2021-06-06 00:00:00,01,04,24,NaN,-75.046404,40.840421


In [30]:
cg17 = cg[cg.Date.dt.year >= 2017]
cg17

,County Code,Municipality Code,Department Case Number,County Name,Municipality Name,Police Department Code,Police Department,Police Station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,Severity,Intersection,Alcohol Involved,HazMat Involved,Crash Type Code,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,SRI (Standard Route Identifier),Mile Post,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,Latitude,Longitude,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,Date,Road Horizontal Alignment,Road Grade,First Harmful Event,reason,lon,lat
4809915,01,01,1708-0494,ATLANTIC,ABSECON CITY,01,ATLANTIC CITY PD,,0,0,0,0,P,B,False,False,01,3,US 30,,30,,00000030__,54.29,02,,02,01,01,01,01,01,1056,FE,E,CR 646 / DELILIAH RD,,,,50,35,NaN,NaN,N,,,2017-08-10 17:35:00,03,04,26,NaN,-74.490831,39.393658
4809916,01,01,1708-0497,ATLANTIC,ABSECON CITY,01,ATLANTIC CITY PD,,0,0,0,0,P,B,False,False,02,2,US 30,W,30,,00000030__,54.29,02,,02,01,01,01,01,05,1056,FE,E,CR 646 / DELILIAH RD,,,,50,35,NaN,NaN,N,,530,2017-08-10 17:36:00,03,04,26,NaN,-74.490831,39.393658
4809917,01,01,1709-1131,ATLANTIC,ABSECON CITY,01,ATLANTIC CITY PD,,0,0,0,0,P,B,False,False,11,1,US 30,W,30,,00000030__,54.29,02,,02,01,06,01,01,01,1056,FE,E,CR 646 / DELILIAH RD,,,,50,35,NaN,NaN,N,,530,2017-09-27 21:27:00,03,04,48,NaN,-74.490831,39.393658
4809918,01,01,1709-1132,ATLANTIC,ABSECON CITY,01,ATLANTIC CITY PD,,0,1,0,0,I,B,False,False,01,2,US 30,W,30,,00000030__,54.29,02,,02,01,06,01,01,05,1056,FE,E,CR 646 / DELILIAH RD,,,,50,35,NaN,NaN,N,,530,2017-09-27 21:28:00,03,04,26,NaN,-74.490831,39.393658
4809919,01,01,1712-0555,ATLANTIC,ABSECON CITY,01,ATLANTIC CITY PD,TRAFFIC,0,0,0,0,P,B,False,False,02,2,US 30,W,30,,00000030__,54.40,02,,02,01,01,01,05,01,,,,,,,,50,,NaN,NaN,N,,630,2017-12-18 13:21:00,01,04,26,NaN,-74.489583,39.392377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,21,23,B150-2021-00117A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,NaN,-75.075607,40.807156
6077299,21,23,B150-2021-00118A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,NaN,-75.056122,40.844345
6077300,21,23,B150-2021-00128A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,NaN,-75.015149,40.863108
6077301,21,23,B150-2021-00130A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,US 46,E,46,,00000046__,7.50,02,,02,01,07,01,05,01,,,,,,,,50,,40.84149,75.04968,N,,8365,2021-06-06 00:00:00,01,04,24,NaN,-75.046404,40.840421


In [32]:
g17 = (~cg17.reason.isna()) | (~cg17.Latitude.isna())
cgg17 = cg17[g17]
cgg17

,County Code,Municipality Code,Department Case Number,County Name,Municipality Name,Police Department Code,Police Department,Police Station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,Severity,Intersection,Alcohol Involved,HazMat Involved,Crash Type Code,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,SRI (Standard Route Identifier),Mile Post,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,Latitude,Longitude,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,Date,Road Horizontal Alignment,Road Grade,First Harmful Event,reason,lon,lat
4809920,01,01,I-2017-000179,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,0,0,0,P,B,False,False,06,2,720 S NEW ROAD,,,,,NaN,09,,02,01,01,01,05,01,,,,,,,,,,NaN,NaN,N,,843,2017-01-03 11:57:00,01,04,28,No MP,NaN,NaN
4809939,01,01,I-2017-002241,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,0,0,0,P,I,False,False,03,2,HOBART AVE,,,,,NaN,07,,02,01,01,01,05,01,,AT,,OAKHURST AVE,,,,25,25,NaN,NaN,N,,846,2017-01-25 16:41:00,01,04,26,No MP,NaN,NaN
4809940,01,01,I-2017-002303,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,0,0,0,P,I,False,False,05,2,CRESTVIEW AVE,,,,,NaN,07,,02,02,01,02,05,01,,AT,,ELM AVE,,,,25,,NaN,NaN,N,,843,2017-01-26 10:05:00,01,04,26,No MP,NaN,NaN
4809948,01,01,I-2017-003718,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,1,0,0,I,B,False,False,06,2,221 NEW JERSEY AVE,,,,,NaN,09,,02,01,01,01,05,01,,,,,,,,,,NaN,NaN,N,OPI CRASHED V1 INTO THE FRONT WALL OF THE BANK...,857,2017-02-08 11:55:00,01,04,28,No MP,NaN,NaN
4809953,01,01,I-2017-004245,ATLANTIC,ABSECON CITY,01,ABSECON CITY PD,,0,1,0,0,I,B,False,False,06,2,CONNECTICUT AVE,,,,,NaN,07,,02,01,01,01,05,01,100,FE,E,OAKHURST AVE,,,,25,25,NaN,NaN,Y,,857,2017-02-13 08:43:00,01,06,26,No MP,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,21,23,B150-2021-00117A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,NaN,-75.075607,40.807156
6077299,21,23,B150-2021-00118A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,NaN,-75.056122,40.844345
6077300,21,23,B150-2021-00128A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,NaN,-75.015149,40.863108
6077301,21,23,B150-2021-00130A,WARREN,WHITE TWP,02,NEW JERSEY STATE POLICE,WASHINGTON - SQ,0,0,0,0,P,B,False,False,12,1,US 46,E,46,,00000046__,7.50,02,,02,01,07,01,05,01,,,,,,,,50,,40.84149,75.04968,N,,8365,2021-06-06 00:00:00,01,04,24,NaN,-75.046404,40.840421
